In [1]:
import test_config as cfg
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import os, os.path, shutil
import pickle
import io

import json

import skimage, skimage.io, skimage.transform, skimage.filters
import sklearn, sklearn.metrics

import importlib

import sys
sys.path.append('../src/')
import modutils
import word_processing as wp
import datetime, time, random

In [2]:
importlib.reload(wp)

<module 'word_processing' from '../src\\word_processing.py'>

In [92]:
%%time
fname = 'D:/Data/bujo_sample_v2/dataset.json'
extraction_path = os.path.join(os.path.dirname(fname),
                               os.path.basename(fname).split('.')[0])
with open(fname, 'r', encoding='utf-8') as f:
    src = json.load(f)

Wall time: 46 ms


In [78]:
src = {k:v for (k,v) in src.items() if k in ['D:\\Data\\bujo_sample_v2\\20190520_232303.jpg', 'D:\\Data\\bujo_sample_v2\\20190520_232309.jpg']}

In [93]:
words = wp.extract_words_from_dataset(src, (1,))
src_images = [skimage.io.imread(os.path.join(extraction_path, x[1]), as_grey=True) for x in words]

In [3]:
transforms_rotation = wp.generate_rotations(-10, 10, 5)
transforms_cutoff = wp.generate_cutoffs(0.5, 0.9, 3)
combinations_all = wp.generate_combinations(transforms_rotation, transforms_cutoff)

fixed_post = [{'type':'trimx'}, {'type':'trimy'}, {'type':'resize', 'y':32}, {'type':'invert'}]
transforms_all = wp.append_transformations(combinations_all, post_transforms=fixed_post)

test_sample = [np.random.choice(np.arange(len(transforms_all)), size=100) for i in range(len(words))]

In [4]:
def convert_word_to_iam(fname):
    sline, sword = fname.split('/')[1].split('.')[0].split('_')
    iline = int(str.replace(sline, 'line', ''))
    iword = int(str.replace(sword, 'word', ''))
    return '{:02d}-{:02d}'.format(iline, iword)

def convert_words_to_iam(fnames, lev0name):
    docs = set(x.split('/')[0] for x in fnames)
    maps = {x:(lev0name, '{:03d}u'.format(i)) for (i, x) in enumerate(docs)}
    res = [(*maps[x.split('/')[0]], convert_word_to_iam(x)) for x in fnames]
    return res

def convert_corpus_to_iam(words, images, transforms, src_path, dst_path):
    if len(transforms) >= 100:
        raise Exception("Too many transforms!")
    names = [z for i in range(len(transforms_all))
             for z in convert_words_to_iam([x[1] for x in words], 'a{:02d}'.format(i))]
    words_path = os.path.join(dst_path, 'words')
    for x in set('/'.join([x[0], '-'.join(x[:2])]) for x in names):
        os.makedirs(os.path.join(words_path, x), exist_ok=True)

    with open(os.path.join(dst_path, 'words.txt'), 'w') as fw:
        for k in range(len(transforms)):
            print(k)
            for (i, x) in enumerate(words):
                j = k * len(words) + i
                fw.write('-'.join(names[j]))
                fw.write(' ok 128 0 0 0 0 NN {}'.format(words[i][0]))
                fw.write('\n')
                tmp_img = wp.perform_transform(src_images[i], transforms[k])
                fname0 = os.path.join(words_path, names[j][0])
                fname1 = os.path.join(fname0, '-'.join(names[j][:2]))
                fname = os.path.join(fname1, '-'.join(names[j]))
                skimage.io.imsave(fname + '.png', tmp_img)

In [94]:
dst_path = 'D:/Data/bujo_iam_valid/'
len(words)

1579

In [95]:
fin_images = [src_images[i] for (i,x) in enumerate(words) if ' ' not in x[0]] 
fin_words = [x for x in words if ' ' not in x[0] and len(x[0]) > 0]

In [96]:
for (i,w) in enumerate(fin_words):
    if len(w[0])==0 or ' ' in w[0]:
        print(w)

In [97]:
charset = open('D:/tmp/TestHTR/model/charList.txt', 'r').read()
charset

'!"(),-.0123456789:?LOPTabcdefhiklmnorstuwyАБВГДЕЖЗИКЛМНОПРСТУФЦЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяё'

In [98]:
def isSubset(word, charset):
    return all(x in charset for x in word)

In [99]:
fin_images = [fin_images[i] for (i,x) in enumerate(fin_words) if isSubset(x[0], charset)]
fin_words = [x for (i,x) in enumerate(fin_words) if isSubset(x[0], charset)]

In [100]:
len(words), len(fin_words)

(1579, 1549)

In [101]:
convert_corpus_to_iam(fin_words, fin_images, transforms_all, extraction_path, dst_path)

0


c:\anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
c:\anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: D:/Data/bujo_iam_valid/words\a00\a00-001u\a00-001u-28-00.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: D:/Data/bujo_iam_valid/words\a00\a00-002u\a00-002u-23-05.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: D:/Data/bujo_iam_valid/words\a00\a00-000u\a00-000u-34-02.png is a low contrast image
  warn('%s is a low contrast image' % fname)


1
2
3


c:\anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: D:/Data/bujo_iam_valid/words\a03\a03-004u\a03-004u-20-00.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: D:/Data/bujo_iam_valid/words\a03\a03-001u\a03-001u-28-00.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: D:/Data/bujo_iam_valid/words\a03\a03-000u\a03-000u-34-02.png is a low contrast image
  warn('%s is a low contrast image' % fname)


4
5
6


...

7
8
9


c:\anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: D:/Data/bujo_iam_valid/words\a09\a09-004u\a09-004u-20-00.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: D:/Data/bujo_iam_valid/words\a09\a09-001u\a09-001u-28-00.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: D:/Data/bujo_iam_valid/words\a09\a09-000u\a09-000u-34-02.png is a low contrast image
  warn('%s is a low contrast image' % fname)


10
11
12


c:\anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: D:/Data/bujo_iam_valid/words\a12\a12-001u\a12-001u-28-00.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: D:/Data/bujo_iam_valid/words\a12\a12-000u\a12-000u-34-02.png is a low contrast image
  warn('%s is a low contrast image' % fname)


13
14
